In [0]:
#all imports
import tensorflow as tf
import torch

In [2]:

gpuname=tf.test.gpu_device_name()
if gpuname=='/device:GPU:0':
  print('Found GPU at :{}'.format(gpuname))
else:
  raise(SystemError('GPU device not found'))

Found GPU at :/device:GPU:0


In [3]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print("There are %d GPU DEVICES available " %torch.cuda.device_count())
  print("The device name is %s"%torch.cuda.get_device_name(0))
else:
  print("No GPU available using only CPU instead")
  device=torch.device("cpu")



There are 1 GPU DEVICES available 
The device name is Tesla P100-PCIE-16GB


In [4]:
!pip install transformers

     |████████████████████████████████| 573kB 5.0MB/s 
     |████████████████████████████████| 890kB 58.3MB/s 
     |████████████████████████████████| 1.0MB 60.3MB/s 
     |████████████████████████████████| 3.7MB 51.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=0d17ae07f23b4b057520df799906f90cbcbe1d30142fffd4658fb68482ffdace
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Custom Data set and Data loader has been inspired from 
#https://github.com/sugi-chan/custom_bert_pipeline/blob/master/bert_pipeline.ipynb

In [0]:
!unzip -P yourpassword -qq '/content/drive/My Drive/DanishData/Danish.zip'

In [0]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [124]:
#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a']
greekdata = pd.read_csv("Danish/offenseval-da-training-v1.tsv", delimiter='\t',names=headers)
data=greekdata[1:]/


SyntaxError: ignored

In [125]:
data=data[:2960]
print(data.head())
print(len(data))

     id  ... Unnamed: 0
0  3131  ...        NaN
1   711  ...        NaN
2  2500  ...        NaN
3  2678  ...        NaN
4   784  ...        NaN

[5 rows x 4 columns]
2960


In [126]:
#GET THE DATA FROM THE PANDAS FRAME
gOFFdata = pd.read_csv("/content/drive/My Drive/translated/TranslatedToDanishFinal.csv", delimiter=',')
print(gOFFdata['subtask_a'])


0       OFF
1       OFF
2       OFF
3       OFF
4       OFF
       ... 
1489    OFF
1490    OFF
1491    OFF
1492    OFF
1493    OFF
Name: subtask_a, Length: 1494, dtype: object


In [127]:
gOFFdata

,Unnamed: 0,Unnamed: 0.1,id,tweet,subtask_a
0,0,1,86426,"@USER @USER Gå hjem, du er fuld !!! @USER #MAG...",OFF
1,1,2,90194,@USER En person burde have taget &quot;dette s...,OFF
2,3,4,62688,@USER @USER Åh nej! Hård lort.,OFF
3,5,6,97670,@USER Canada har ikke brug for endnu et CUCK! ...,OFF
4,6,7,77444,@USER du er en liggende korrupt forræder !!! I...,OFF
...,...,...,...,...,...
1489,4377,4378,86690,"@USER Gå til Iran, hvor du hører til dum idiot.",OFF
1490,4379,4380,58201,@USER @USER @USER @USER Du skal vide bedre at ...,OFF
1491,4381,4382,12178,@USER denne jag off ref Tony Corrente har brug...,OFF
1492,4395,4396,96924,@USER @USER @USER @USER Løgnere som Antifa-tvi...,OFF


In [0]:
data=pd.concat([data, gOFFdata], ignore_index=True)

In [141]:
data.to_numpy()[:,2]

array(['NOT', 'NOT', 'OFF', ..., 'OFF', 'OFF', 'OFF'], dtype=object)

In [0]:

dfnumpy=data.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)


In [131]:
print(len(data))

4454


In [132]:
pip install emoji --upgrade

Requirement already up-to-date: emoji in /usr/local/lib/python3.6/dist-packages (0.5.4)


In [0]:
import re

def ft1(l):
    a = re.split('|\]|\[|\)|\(|; |,|\*|\n',l)
    li = []
    for t in a:
        li.append(t.strip(' ,')) 
    return li

In [0]:
#Removing duplicate words inspired from https://stackoverflow.com/questions/57424661/how-to-efficiently-remove-consecutive-duplicate-words-or-phrases-in-a-string

In [0]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
import emoji
import string
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
def preprocessinglib(arrt):
    ans =[]
    for txt in arrt:
        txt1=emoji.demojize(txt)
        unique_words = dict.fromkeys(txt1.split(" "))
        txt2=' '.join(unique_words)
        ans.append(ft1(txt2))
               
    return ans

In [0]:
arrt=x[:,0]

In [147]:
allTokens=preprocessinglib(arrt)

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [0]:
#len(allTokens)
preprocessedTweets=allTokens

In [0]:

from sklearn.utils import shuffle
preprocessedTweets, y = shuffle(preprocessedTweets, y)

In [151]:
print(preprocessedTweets[0:5])

[['@USER Hun siger, at hun er dybt religiøs &quot;... sagde ikke&quot; hvilken &quot;religion ... satanister meget religiøse Fred&quot;'], ['Prol'], ['Lel hvor har du fundet den? Det er vidst det jeg ville kalde et tamt meme.'], ['@USER du er en beskidt cum dumpster'], ['@USER Åh herregud. De er så skide søde.']]


In [152]:
from transformers import BertTokenizer as bertTokenizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

Using TensorFlow backend.


In [153]:
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)


In [0]:
ids_of_sentence=[]
ids_of_sentence_words=[]
attention_masks=[]
def giveIds(sentence,y_):
  ids_of_sentence=[]
  ids_of_sentence_words=[]
  attention_masks=[]
  maxlength=0
  for t in sentence:
      tokenized_sentence_id=tokenizer.encode(t,add_special_tokens=True)
      if(maxlength<len(tokenized_sentence_id)):
          maxlength=len(tokenized_sentence_id)
      ids_of_sentence.append(tokenized_sentence_id)
  print(maxlength)
  ids_of_sentence_words=pad_sequences(ids_of_sentence,maxlen=maxlength,dtype="long",value=0,truncating="post",padding="post")##can change max length
  attention_masks = [[int(a > 0)   for a in b ]for b in ids_of_sentence_words] 
  #print(len(attention_masks))
  #print(len(ids_of_sentence_words))
  return ids_of_sentence_words,attention_masks

In [0]:
sentence_train, sentence_test, y_train, y_test = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42)
#x_train_mask,x_test_mask,_,_=train_test_split(attention_masks,y, test_size=0.2, random_state=42)



In [0]:
##do this at the time of training only 
#sentence_train, sentence_test1, y_train, y_test1 = train_test_split(preprocessedTweets,y, test_size=0.2, random_state=42) only this for code submission
#sentence_test,sentence_predict, y_test,y_predict = train_test_split(sentence_test1,y_test1, test_size=0.5, random_state=42)
#x_predict_mask,_,_,_=train_test_split(x_test_mask,y_test, test_size=0.5, random_state=42)

In [157]:
y_test

array([['NOT'],
       ['OFF'],
       ['NOT'],
       ...,
       ['OFF'],
       ['OFF'],
       ['NOT']], dtype=object)

In [158]:

from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

print(y_train.shape)
print(y_test.shape)
yTrain=le.fit_transform(y_train.flatten())
print(yTrain.shape)
print(le.classes_)


(4758, 1)
(1190, 1)
(4758,)
['NOT' 'OFF']


In [0]:
yTest=le.transform(y_test.flatten())


In [160]:
set(y_test.flatten())

{'NOT', 'OFF'}

In [161]:
set(yTest)

{0, 1}

In [0]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTrainDataset(Dataset):
    def __init__(self,xytrain):
        self.xytrain = xytrain
        self.maxlength=64
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytrain[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_train_pytorch = torch.tensor(ids_of_sentence_word)
        y_train_pytorch=torch.tensor(self.xytrain[1][index])
        x_train_mask_pytorch=torch.tensor(attention_mask)
        
        return x_train_pytorch,x_train_mask_pytorch,y_train_pytorch
        
        
    def __len__(self):
        return len(self.xytrain[0])

In [0]:
#from torch.utils.data import Dataset
#tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekTestDataset(Dataset):
    def __init__(self,xytest):
        self.xytest = xytest
        self.maxlength=64
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xytest[0][index]))
        if len(tokenized_review) > self.maxlength:
            #print(tokenized_review)
            tokenized_review = tokenized_review[:self.maxlength-2]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_test_pytorch = torch.tensor(ids_of_sentence_word)
        y_test_pytorch=torch.tensor(self.xytest[1][index])
        x_test_mask_pytorch=torch.tensor(attention_mask)
        
        return x_test_pytorch,x_test_mask_pytorch,y_test_pytorch
        #return [1,2,3]
        
    def __len__(self):
        return len(self.xytest[0])

In [0]:
xytrain=[sentence_train,yTrain]
tdataset = GreekTrainDataset(xytrain)
tsampler=RandomSampler(tdataset)
tdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)

In [0]:
xytest=[sentence_test,yTest]
tedataset = GreekTestDataset(xytest)
tesampler=RandomSampler(tedataset)
tedataloader = DataLoader(tedataset, batch_size=32, num_workers=1, shuffle=False,sampler=tesampler)

In [167]:
from transformers import BertForSequenceClassification as bfsc,AdamW,BertConfig
model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [168]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/DanishData/augdanish.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/DanishData/augdanish.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [0]:
params=list(model.named_parameters())

In [0]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
{
"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
"weight_decay": 0.01,
},
{"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

In [0]:
optimizer=AdamW(model.parameters(),lr=2e-5,eps=1e-8)
#############CAN CHANGE LEARNING RATE 

In [0]:
from transformers import get_linear_schedule_with_warmup

epochs=4
total_steps=len(tdataloader)*epochs

sch=get_linear_schedule_with_warmup(optimizer,
                                    num_warmup_steps=0,num_training_steps=total_steps)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  labelsFlattend=labels.flatten()
  print("predictionArgmax",predictionArgmax)
  print("labelsFlattend",labelsFlattend)
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
#Also based on the following tutorials
#https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [174]:
import random
import time 

def set_seed(seed,ngpu):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if ngpu > 0:
        torch.cuda.manual_seed_all(seed)
      
set_seed(42,torch.cuda.device_count())
#remove later

epochs=4
lossList=[]
max_grad_norm=1.0
for e in range(0, epochs):
    print("Start Epoch Number",(e + 1))
    print("Start Training")
    
    #Amount of time taken for training
    t1 = time.time()
    tr_loss, logging_loss = 0.0, 0.0
    model.train()
    tsteps=0
    for step, batch in enumerate(tdataloader):
        if step % 50 == 0 and not step == 0:
            print("Batch Completed  {:,}  of  {:,}.    Elapsed time is  {}".format(step, len(tdataloader),time.time() - t1))
        
        batch = tuple(t.to(device) for t in batch)
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        model.zero_grad()        
        outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"], labels=inputs["labels"])

        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        tsteps+=1
        optimizer.step()
        sch.step()

    a_tr_loss = tr_loss /(tsteps)               
    lossList.append(a_tr_loss)
    
    print(" The training loss incured is  {0:.3f}".format(a_tr_loss))
    t2=time.time()
    print("  Training one epoch time taken",t2-t1)
    print(" Validation starts here ")
   
    t1 = time.time()
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    eval_f1=0
    eval_acc=0

    for batch in tedataloader:       
        batch = tuple(t.to(device) for t in batch)        
        inputs = {"input_ids": batch[0], "attention_mask": batch[1], "labels": batch[2]}
        with torch.no_grad():        
            outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = (inputs["labels"]).to('cpu').numpy()
        tmpf1score,tmpaccscore = calculateF1Score(logits, label_ids)
        eval_f1 = eval_f1+tmpf1score
        eval_acc=eval_acc+tmpaccscore
        nb_eval_steps += 1
        #print(" TEMP F1 score: {0:.3f}".format(tmpf1score))
        #print("TEMP  Accuracy score: {0:.3f}".format(tmpaccscore))

    torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/DanishData/augdanish.pth.tar')
    print("  F1 score: {0:.3f}".format(eval_f1/nb_eval_steps))
    print("  Accuracy score: {0:.3f}".format(eval_acc/nb_eval_steps))
    
    t2=time.time()
    print("  Validating one epoch time taken ",t2-t1)
    
print("ALL DONE!!!")

Start Epoch Number 1
Start Training
Batch Completed  50  of  149.    Elapsed time is  11.724251508712769
Batch Completed  100  of  149.    Elapsed time is  23.150556802749634
 The training loss incured is  0.293
  Training one epoch time taken 34.294397592544556
 Validation starts here 
predictionArgmax [1 1 0 1 1 1 1 1 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0 1 1 0 1 0 0 1 1 1]
labelsFlattend [1 1 0 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 0 1 0 0 1 1 1]
predictionArgmax [1 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 0 0 1 0]
labelsFlattend [1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 0]
predictionArgmax [0 1 0 0 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1]
labelsFlattend [0 1 0 0 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1]
predictionArgmax [0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 0 0]
labelsFlattend [1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 0 1 0 0]
predictionArgmax [0 0 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 

In [175]:
!unzip -P yourpassword -qq '/content/drive/My Drive/DanishData/DanishPredict.zip'
!unzip -P yourpassword -qq '/content/drive/My Drive/DanishData/Danish.zip'

replace Danish/offenseval-da-training-v1.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Danish/readme-trainingset-da.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
ygiven=[]
ypredicted=[]

def convertToInt(val):
    if not val:
        return 0    
    try:
        return np.int64(val)
    except:        
        return np.int64(0)




In [0]:
##Use blob
#GET THE DATA FROM THE PANDAS FRAME
def readData1():
  headers=['id','tweet']
  greekDataTest = pd.read_csv("offenseval-da-test-v1-nolabels.tsv", delimiter='\t',names=headers,quoting=csv.QUOTE_NONE,
                                converters={"id":convertToInt})
  greekDataTest=greekDataTest[1:]
  print(greekDataTest.head())
  print(greekDataTest.dtypes)
  print(greekDataTest.shape)
  
  dfnumpy=greekDataTest.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  tid=dfnumpy[:, 0].reshape(-1, 1)
  print(tid)
  arrt=X[:,0]
  #allTokens=preprocess1(arrt)
  preprocessedTweets=arrt
  return preprocessedTweets,tid

In [178]:
preprocessedTweets,tid=readData1()


     id                                              tweet
1  1382  Der er syriske "flygtninge" som rejser til Ira...
2  1384                             Danmark = Vitryssland?
3   547  Ja tvangsfjernelser af børn på urigtige oplysn...
4  1269  Han kan ikke Svensk og forventer et job. Hvis ...
5  1695                                  NED MED SVENSKEN!
id        int64
tweet    object
dtype: object
(329, 2)
[[1382]
 [1384]
 [547]
 [1269]
 [1695]
 [2320]
 [990]
 [2665]
 [1414]
 [262]
 [1459]
 [3375]
 [1431]
 [137]
 [8]
 [2964]
 [2542]
 [1375]
 [1458]
 [799]
 [976]
 [305]
 [2643]
 [386]
 [715]
 [2041]
 [2194]
 [60]
 [2181]
 [720]
 [552]
 [2560]
 [775]
 [848]
 [1754]
 [2363]
 [1330]
 [885]
 [974]
 [1205]
 [2589]
 [2172]
 [159]
 [3425]
 [544]
 [1371]
 [30]
 [1973]
 [22]
 [1813]
 [1379]
 [99]
 [1614]
 [1038]
 [633]
 [1275]
 [924]
 [962]
 [1314]
 [926]
 [33]
 [2627]
 [3396]
 [1321]
 [2321]
 [391]
 [1143]
 [1596]
 [3222]
 [3122]
 [716]
 [173]
 [2382]
 [3560]
 [2046]
 [1291]
 [2858]
 [102]
 [257

In [0]:
from torch.utils.data import Dataset
tokenizer=bertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=True)
class GreekPredictDataset(Dataset):
    def __init__(self,xypredict):
        self.xypredict = xypredict
        self.maxlength=128
       
    def __getitem__(self, index):
        tokenized_review = tokenizer.tokenize(str(self.xypredict[0][index]))
        if len(tokenized_review) > self.maxlength:
            tokenized_review = tokenized_review[:self.maxlength]
        
        
        ids_of_sentence_word  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (self.maxlength - len(ids_of_sentence_word))
        ids_of_sentence_word += padding
        assert len(ids_of_sentence_word) == self.maxlength
        #print(ids_of_sentence_word)
        attention_mask = [int(b > 0) for b in ids_of_sentence_word] 
        x_predict_pytorch = torch.tensor(ids_of_sentence_word)
        x_predict_mask_pytorch=torch.tensor(attention_mask)
        tid_predict_pytorch=torch.tensor(self.xypredict[1][index])
        
        return x_predict_pytorch,x_predict_mask_pytorch,tid_predict_pytorch
       
    def __len__(self):
        return len(self.xypredict[0])
 


In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
def calculateScore(predictions):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).flatten()
  yres.append(predictionArgmax.flatten())
  print("predictionArgmax",predictionArgmax)
 

In [0]:
yres=[]
finalTid=[]
def predictingData(pTweets,tid):
  #https://colab.research.google.com/drive/1Y4o3jh3ZH70tl6mCd76vz_IxX23biCPP#scrollTo=1M296yz577fV
  ids_of_sentence=[]
  predictedLabels,trueLabels=[],[]
  
  map_location=""
  xypredict=[pTweets,tid.flatten()]
  
  tdataset = GreekPredictDataset(xypredict)
  tsampler=RandomSampler(tdataset)
  predictdataloader = DataLoader(tdataset, batch_size=32, num_workers=1, shuffle=False,sampler=tsampler)
  print(device.type)
  '''model=bfsc.from_pretrained('bert-base-multilingual-cased',num_labels=2,output_attentions=False,output_hidden_states=False)
  if device.type=="cpu":
    model.to(device)
    map_location='cpu'
  else:
    model.cuda()
    map_location=lambda storage, loc: storage.cuda()
  params=list(model.named_parameters())
  eval_f1=0
  eval_acc=0
  nb_eval_steps=0
  checkpoint = torch.load('/content/drive/My Drive/DanishData/augdanish.pth.tar',map_location=map_location)
  print("Hello")
  model.load_state_dict(checkpoint['state_dict'])'''
  model.eval()
  i=0
  for batch in predictdataloader:
      print(i)
      i=i+1
      batch = tuple(t.to(device) for t in batch)        
      inputs = {"input_ids": batch[0], "attention_mask": batch[1], "tids":batch[2]}
      
      with torch.no_grad():       
          outputs = model(inputs["input_ids"],token_type_ids=None,attention_mask=inputs["attention_mask"])
      logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      predictedLabels.append(logits)
      tidl=(inputs["tids"]).to('cpu').numpy()
      finalTid.append(tidl)
      calculateScore(logits)
      
  return predictedLabels,finalTidF1

In [0]:
preprocessedTweets,tid=readData1()

In [0]:
predictedLabels,finalTid=predictingData(preprocessedTweets,tid)

In [0]:

torch.save({'state_dict': model.state_dict()}, '/content/drive/My Drive/DanishData/augdanish.pth.tar')
checkpoint = torch.load('/content/drive/My Drive/DanishData/augdanish.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

In [0]:
yans=[yres[i].flatten().tolist() for i in range(len(yres))]
ytid=[finalTid[i].flatten().tolist() for i in range(len(finalTid))]

In [0]:
from itertools import chain

yans=list(chain.from_iterable(yans))
ytid=list(chain.from_iterable(ytid))

In [0]:
len(ytid)

In [0]:
yans1=["NOT" if yans[i]==0 else "OFF" for i in range(len(yans))]

In [0]:
print(yans1.count("OFF"))
print(yans1.count("NOT"))

In [0]:
#z={'tweet':sentence_predict[],'subtask_a':y_predict}
#print(len(z))
C = {'id': ytid,
        'predicted': yans1,
    }
df = pd.DataFrame(C)
print (df[df['id'] == 400] )

export_csv = df.to_csv ('/content/drive/My Drive/DanishData/Answer.csv', index = None, header=False)

from 80 ,10 ,10 split getting  # F1 score: 0.836 Accuracy score: 0.867